In [65]:
### Import de bibliotecas para manipulação dos dados
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer

In [66]:
#Caminho e leitura do Dataframe
df = pd.read_csv('../data/raw/Orange_Quality_Data.csv')

#Análise das 10 primeiras linhas 
df.head(10)

,Size (cm),Weight (g),Brix (Sweetness),pH (Acidity),Softness (1-5),HarvestTime (days),Ripeness (1-5),Color,Variety,Blemishes (Y/N),Quality (1-5)
0,7.5,180,12.0,3.2,2.0,10,4.0,Orange,Valencia,N,4.0
1,8.2,220,10.5,3.4,3.0,14,4.5,Deep Orange,Navel,N,4.5
2,6.8,150,14.0,3.0,1.0,7,5.0,Light Orange,Cara Cara,N,5.0
3,9.0,250,8.5,3.8,4.0,21,3.5,Orange-Red,Blood Orange,N,3.5
4,8.5,210,11.5,3.3,2.5,12,5.0,Orange,Hamlin,Y (Minor),4.5
5,6.7,126,9.1,3.0,2.0,25,2.0,Orange,Navel,N,1.0
6,7.2,160,9.0,3.5,3.5,9,4.0,Yellow-Orange,Tangelo (Hybrid),N,4.0
7,6.5,130,13.5,2.8,1.5,5,4.5,Light Orange,Murcott (Hybrid),N,4.5
8,8.8,240,7.5,4.0,5.0,18,3.0,Deep Orange,Moro (Blood),Y (Sunburn),3.0
9,7.8,190,12.0,3.1,2.0,11,4.5,Orange,Jaffa,N,5.0


In [67]:
binario = df['Blemishes (Y/N)']
binario.head(30)

0                     N
1                     N
2                     N
3                     N
4             Y (Minor)
5                     N
6                     N
7                     N
8           Y (Sunburn)
9                     N
10        Y (Mold Spot)
11        Y (Mold Spot)
12                    N
13                    N
14                    N
15                    N
16           Y (Bruise)
17          Y (Sunburn)
18                    N
19                    N
20                    N
21                    N
22                    N
23       Y (Split Skin)
24                    N
25                    N
26                    N
27                    N
28                    N
29    Y (Sunburn Patch)
Name: Blemishes (Y/N), dtype: object

In [68]:
# Caminho do dicionário dos dados .csv
csv_path = '../data/external/dicionario_de_dados_laranjas.csv'

# Leitura do CSV
df_dicionario_dados = pd.read_csv(csv_path, sep=',')
#dicionario = df_dicionario_dados.loc[:, ~df_dicionario_dados.columns.str.contains('^Unnamed')]
print(df_dicionario_dados)

             Variável           Tradução   \
0            Size (cm)            Tamanho   
1           Weight (g)               Peso   
2     Brix (Sweetness)             Doçura   
3         pH (Acidity)             Acidez   
4            Softness              Maciez   
5   HarvestTime (days)  Tempo de colheita   
6            Ripeness           Maturação   
7                Color                Cor   
8           Blemishes             Manchas   
9             Quality           Qualidade   
10             Variety          Variedade   

                                           Descrição   \
0                  Diâmetro da laranja em centímetros   
1                           Peso da laranja em gramas   
2                          Nível de açúcar da laranja   
3                         Nível de acidez da laranja    
4   Nível de maciez da laranja em uma escala de 1 a 5   
5       Número de dias desde o plantio até a colheita   
6   Nível de maturação da laranja em uma escala de...   
7  

In [69]:
print(df_dicionario_dados.columns)

Index(['Variável ', 'Tradução ', 'Descrição ', 'Espectro de opções', 'Tipo ',
       'Subtipo ', 'Unnamed: 6', '           '],
      dtype='object')


In [70]:
#Organização da busca pelo tipo de variável

variavel_alvo = 'Quality'

# Renomeando colunas (opcional)
df_dicionario_dados = df_dicionario_dados.rename(columns={'Variável ': 'Variavel'})
df_dicionario_dados = df_dicionario_dados.rename(columns={'Subtipo ': 'Subtipo'})

# Acessando a coluna
variavel_Binaria = df_dicionario_dados['Variavel']

variavel_Binaria = (
    df_dicionario_dados
    .query("Subtipo == 'Binária' and Variavel != @variavel_alvo")
    .Variavel.to_list()
)
print(f"Variáveis qualitativo: Binária")
print(variavel_Binaria)
print(" ")

variavel_nominal = (
    df_dicionario_dados
    .query("Subtipo == 'Nominal' and Variavel != @variavel_alvo")
    .Variavel
    .to_list()
)
print(f"Variáveis qualitativo: nominal")
print(variavel_nominal)
print(" ")

variavel_quanti_continua = (
    df_dicionario_dados
    .query("Subtipo == 'Contínua' and Variavel != @variavel_alvo")
    .Variavel #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Contínua")
print(variavel_quanti_continua)
print(" ")
variavel_quanti_discreta = (
    df_dicionario_dados
    .query("Subtipo == 'Discreto' and Variavel != @variavel_alvo")
    .Variavel #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Discreto")
print(variavel_quanti_discreta)
print(" ")

#X = df.drop(columns=[variavel_alvo])
#Y = df[variavel_alvo]

Variáveis qualitativo: Binária
['Blemishes ']
 
Variáveis qualitativo: nominal
['Color', 'Variety']
 
Variável quantitativo: Contínua
['Size (cm)', 'Weight (g)', 'Brix (Sweetness)', 'pH (Acidity)']
 
Variável quantitativo: Discreto
[]
 


In [71]:
processamento_nominal = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), # tratamento para dados faltantes
    ("encoding", OneHotEncoder(sparse_output=False)), # codificação de variáveis
])

processamento_ordinal = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), #Tratamento de dados faltantes com substituição pela MODA
    ("encoding", OrdinalEncoder()), #Codificação de variáveis
     
])

processamento_continuos = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')), #Tratamento de dados faltantes com substituição pela MÉDIA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])
processamento_discreta = Pipeline(steps=[
    ("missing", KNNImputer()), #Tratamento de dados faltantes com substituição pela MEDIANA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])
processamento_binario = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')),
    ("encoding", OneHotEncoder(sparse_output=False))
])


In [72]:
pre_processador = ColumnTransformer([
    ("continuo", processamento_continuos, variavel_quanti_continua),
    ("discreta", processamento_discreta, variavel_quanti_discreta),
    ("binario", processamento_binario, variavel_Binaria)
])

## K-fold (Validação Cruzada)

In [104]:
#k-fold
# Create a Pipeline
model = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False)),  # Optional scaler for numerical features
    ('regressor', LinearRegression())
])

# Perform cross-validation
scores = cross_val_score(model, df, y, cv=10, scoring='neg_mean_squared_error')

print(scores)

mean_score = np.mean(scores)
std_score = np.std(scores)

print("Média:", mean_score)
print("Desvio Padrão:", std_score)

[-0.2641057  -0.26356242 -0.20066304 -0.16862867 -0.51679708 -0.45397963
 -0.59602327 -0.32778338 -0.56468708 -0.33432058]
Média: -0.3690550849311918
Desvio Padrão: 0.1457415016434597


#### Como é possivel observar, foram usados 10 folds Os resultados da validação cruzada indicaram um valor de -0.36 de MSE, sugerindo que o modelo é capaz de fazer previsões com uma precisão razoável. O Desvio padrão 0.14 do MSE demonstra a robustez do modelo em diferentes subconjuntos dos dados.